### Задача о потоке пар чисел.
Даны N пар чисел. Из каждой паре берется по твари и подсчитываются суммы такого рода. Необходимо 
* найти все возможные суммы
* найти максимальную сумму, не делящуюся на 5, или вывести -1 при отсутствии таковой
* организовать перебор и найти оптимальный алгоритм 

#### Перебор с помощью перебора битовых масок всех подмножеств

In [1]:
# создадим _генератор_ всех возможных подмножеств множества n элементов:
# [True, False, True] -> Значит, первый и третий элемент принадлежат подмножеству
# в простом случае, можно генератор заменить на функцию, возвращающую список списков

def subset_indecies(n):
    """ Генерирует последовательность из списков вида [1, 0, 0, 1, 1 ...]
        Всего таких списков 2^n, каждый отвечает одному подмножеству из n элементов:
            принадлежит ли i элемент полного множества данноому подмножеству.
    """
    all_subset_idecies = []
    for mask in range(1 << n):
        # bool(expr) превратит 0 в False, остальные числа в True
        # int(bool(expr)) превращает нули в нули, все остальное в 1
        all_subset_idecies.append([int(bool((1 << i) & mask)) for i in range(n)])
    return all_subset_idecies

def get_all_sums_by_mask(pairs):
    summs = []
    for indecies in subset_indecies(len(pairs)):
        
        # для справки, _можно понимать_ действие zip как 
        # zip([1,2,3], [a, b, c]) -> [(1, a), (2, b), (3, c)]
        # например,
        # indecies = [1, 0, 0, 1]
        # pairs = [[1,2], [3,4], [5,6], [7,8]]
        # >>> choosed_elements = [2, 3, 5, 8]
        
        choosed_elements = [pair[int(i)] for i, pair in zip(indecies, pairs)]
        summs.append((sum(choosed_elements), ''.join(str(i) for i in indecies)))
    
    return summs


        
        
        

In [2]:
pairs = [[6, 11], [4, 2], [5, 10]]

get_all_sums_by_mask(pairs)

[(15, '000'),
 (20, '100'),
 (13, '010'),
 (18, '110'),
 (20, '001'),
 (25, '101'),
 (18, '011'),
 (23, '111')]

#### Перебор с помощью древовидной рекурсии



In [3]:
pairs = [[6, 11], [4, 2], [5, 10]]

In [4]:
def get_all_sums_recursive(pairs):
    sums = []
    def helper(level, acc, path):
        if level == len(pairs):
            sums.append((acc, path))
        else:
            helper(level + 1, acc + pairs[level][0], path + '0')
            helper(level + 1, acc + pairs[level][1], path + '1')
    helper(0, 0, '')
    return sums

In [5]:
get_all_sums_recursive(pairs)

[(15, '000'),
 (20, '001'),
 (13, '010'),
 (18, '011'),
 (20, '100'),
 (25, '101'),
 (18, '110'),
 (23, '111')]

Объединим эти функции перебора в решении, которое может использовать любую из них (по умолчанию используя второй вариант):

In [6]:
def bf_solution(pairs, sums_generator_func=get_all_sums_recursive):
    return max([s for s in sums_generator_func(pairs) if s[0] % 5 != 0], default=[-1, ""])

In [7]:
print(bf_solution(pairs))  # recursive by default
print(bf_solution(pairs, sums_generator_func=get_all_sums_by_mask))

(23, '111')
(23, '111')


Попробуем для большего количества пар:

In [8]:
pairs_6 = [[1,2], [3,4], [2,6], [11,3], [5, 9], [2, 8]]

In [9]:
get_all_sums_recursive(pairs_6)

[(24, '000000'),
 (30, '000001'),
 (28, '000010'),
 (34, '000011'),
 (16, '000100'),
 (22, '000101'),
 (20, '000110'),
 (26, '000111'),
 (28, '001000'),
 (34, '001001'),
 (32, '001010'),
 (38, '001011'),
 (20, '001100'),
 (26, '001101'),
 (24, '001110'),
 (30, '001111'),
 (25, '010000'),
 (31, '010001'),
 (29, '010010'),
 (35, '010011'),
 (17, '010100'),
 (23, '010101'),
 (21, '010110'),
 (27, '010111'),
 (29, '011000'),
 (35, '011001'),
 (33, '011010'),
 (39, '011011'),
 (21, '011100'),
 (27, '011101'),
 (25, '011110'),
 (31, '011111'),
 (25, '100000'),
 (31, '100001'),
 (29, '100010'),
 (35, '100011'),
 (17, '100100'),
 (23, '100101'),
 (21, '100110'),
 (27, '100111'),
 (29, '101000'),
 (35, '101001'),
 (33, '101010'),
 (39, '101011'),
 (21, '101100'),
 (27, '101101'),
 (25, '101110'),
 (31, '101111'),
 (26, '110000'),
 (32, '110001'),
 (30, '110010'),
 (36, '110011'),
 (18, '110100'),
 (24, '110101'),
 (22, '110110'),
 (28, '110111'),
 (30, '111000'),
 (36, '111001'),
 (34, '111010'

In [10]:
bf_solution(pairs_6)

(39, '101011')

#### Оптимальное решение

* Временн**а**я сложность: O(n)
* Расход памяти: O(1)

"Жадный" алгоритм:

Выбирая в каждой паре максимальное число, получим, очевидно, максимально возможную сумму.

Например, дан список пар:
```
3 11
4 2
2 10
```

Максимальная сумма 11 + 4 + 10 = 25. Но она предательски делится на 5 :( Что же делать?

Попробуем найти _замену_ одному из чисел в максимальной сумме - его пару. Очевидно, что если остатки от деления на 5 в паре одинаковы, то менять смысла нет, сумма все равно будет делиться на 5. Поэтому менять нужно только в том случае, если в паре числа имеют _разные остатки от деления на 5_. Но какое же выбрать? Попробуем быть "жадными".

*Первая гипотеза*: выберем максимальное из вторых чисел. Тогда менять нужно первую пару. 11 садится на скамейку, 3 выходит на площадку. И команда тут же проигрывает :((((

3 + 4 + 10 = 17, на 5 не делится, да. Но есть ведь сумма и побольше: 11 + 2 + 10 = 22. Внимательно вглядываясь в эти числа, находим верное решение:

*Вторая гипотеза*: выберем такое из подходящих по остатку, что наша сумма уменьшится слабее всего, т.е. второе число _с наименьшей разностью с первым_. Тогда как раз и получится команда-победитель!


##### Доказательство алгоритма

Наши рассуждения красивы и очень правдоподобны, а простейшие ручные эксперименты на небольших наборах данных не смогли опровергнуть наш алгоритм. Именно так: не доказали, а лишь не смогли опровергнуть, что, правда, даёт нам повод для оптимизма в поисках *доказательства*.

Предположим, что мы несколько по-иному выбирали числа из пар и нашли сумму, которая не делится на 5, при этом больше полученной по нашему алгоритму. Тогда от максимальной суммы она отличается выбором замен минимум в 2х парах, так как если ограничить замены от максимальной суммы только одной парой, то наш алгоритм очевидно верен.
Но тогда тоже сумма разностей во всех замененных парах будет больше одной и минимальной разности. Выходит, что эта другая сумма никак не может быть больше нашей.


In [11]:
# эта реализация не рассказывает о выборе элементов пар. это остаётся в качестве упражнения для падаванов)

def optimal(pairs):
    substitution_fst = -1
    substitution_snd = -1
    summa = 0
    
    for a, b in pairs:
        a, b = max(a, b), min(a, b) # парное присваивание, особенность python
        summa += a
        if a % 5 != b % 5:
            if (a - b < substitution_fst - substitution_snd) or substitution_fst == -1:
                substitution_fst, substitution_snd = a, b
    
    if summa % 5 == 0:
        if substitution_fst != -1:
            summa -= substitution_fst
            summa += substitution_snd
        else:
            summa = -1
    
    return summa
    

In [12]:
optimal([[1, 2]])

2

In [13]:
optimal(pairs)

23

In [14]:
optimal(pairs_6)

39

### Test

Протестируем равенство ответов переборного и оптимально алгоритма:

In [16]:
import random

data = [
    [[1,1], [1,1], [1,1], [1,1], [1,1]],
    [[1,2], [3,4], [5,6]],
    [[1,2], [4,9], [11, 10]],
    [[3, 8], [12, 2]],
]

def makepairs():
    n = random.randint(5, 13)  # нельзя брать слишком большие списки для перебора, сложность O(2^n)
    return [[random.randint(1, 1000), random.randint(1, 1000)] for i in range(n)]

data.extend([makepairs() for i in range(100)])
    
    

for ds in data:
    ans_bf = bf_solution(ds)[0]
    ans_opt = optimal(ds)
    if ans_bf != ans_opt:
        print("FAILED:", ds, ans_bf, ans_opt)
        break
else:
    print("TEST PASSED")

TEST PASSED
